In [27]:
import pmagpy.new_builder as nb
from pmagpy import ipmag
import os
import json
import numpy as np
import sys

import pandas as pd
from pandas import DataFrame
from pmagpy import builder
from pmagpy import validate_upload
from pmagpy import pmag
from SPD.mapping import map_magic

reload(nb)
reload(map_magic)

pmagpy_dir = os.path.join(os.getcwd(), 'pmagpy')
if pmagpy_dir not in sys.path:
    sys.path.append(pmagpy_dir)
    

WD = os.path.join(os.getcwd(), "2_5", "McMurdo")

In [16]:
# convert magic_measurements to measurements (3.0)
# first unpack lawrence et al., 2009 datafile from MagIC 

!download_magic.py -f zmab0100049tmp03.txt -WD 2_5/McMurdo -ID 2_5/McMurdo

['f', 'F', 'A', 'WD', 'ID', 'Fsa', 'Fsi'] 7
-I- using default for arg: F
-
-I- using default for arg: A
-
-I- using default for arg: Fsa
-
-I- using default for arg: Fsi
-
-I- using default for arg: O
-
2_5/McMurdo
1  records written to file  2_5/McMurdo/er_locations.txt
140  records written to file  2_5/McMurdo/er_sites.txt
1418  records written to file  2_5/McMurdo/er_samples.txt
1046  records written to file  2_5/McMurdo/er_specimens.txt
99  records written to file  2_5/McMurdo/er_ages.txt
31  records written to file  2_5/McMurdo/er_citations.txt
4  records written to file  2_5/McMurdo/er_mailinglist.txt
431  records written to file  2_5/McMurdo/er_images.txt
25470  records written to file  2_5/McMurdo/magic_measurements.txt
1313  records written to file  2_5/McMurdo/pmag_specimens.txt
207  records written to file  2_5/McMurdo/pmag_sites.txt
177  records written to file  2_5/McMurdo/pmag_results.txt
8  records written to file  2_5/McMurdo/pmag_criteria.txt
10  records written to fil

In [18]:
# read in data model 2.5 measruements file
data2,filetype= pmag.magic_read(WD+'/magic_measurements.txt') 
print filetype, len(data2)

magic_measurements 25470


In [9]:
def flip_dict(dict):
    newdict={}
    for key in dict:
        val=dict[key]
        newdict[val]=key
    return newdict

In [25]:
spec3_spec2={'int_drats': 'specimen_drats', 'site': 'er_site_name', 'int_mad': 'specimen_int_mad', 'sample': 'er_sample_name', 'measurement_step_max': 'meas_step_max', 'specimen_n': 'dir_n_measurements', 'int_n_measurements': 'specimen_int_n', 'int_corr': 'specimen_correction', 'int_rsc': 'specimen_rsc', 'analyst_names': 'er_analyst_mail_names', 'int_scat': 'specimen_scat', 'int_ptrm_n': 'specimen_int_ptrm_n', 'citations': 'er_citation_names', 'int_gmax': 'specimen_gmax', 'int_dang': 'specimen_int_dang', 'dir_tilt_correction': 'specimen_tilt_correction', 'location': 'er_location_name', 'dir_comp': 'specimen_comp_name', 'specimen_magn_moment': 'magn_moment', 'int_w': 'specimen_w', 'specimen': 'er_specimen_name', 'int_q': 'specimen_q', 'int_fvds': 'specimen_fvds', 'specimen_mad': 'dir_mad_free', 'int_frac': 'specimen_frac', 'meas_step_min': 'measurement_step_min', 'int_f': 'specimen_f', 'software_packages': 'magic_software_packages', 'dir_mad_free': 'specimen_mad', 'magn_moment': 'specimen_magn_moment', 'instrument_codes': 'magic_instrument_codes', 'int_b_beta': 'specimen_b_beta', 'dir_n_comps': 'specimen_comp_n', 'int_md': 'specimen_md', 'dir_n_measurements': 'specimen_n', 'dir_inc': 'specimen_inc', 'specimen_magn_volumn': 'magn_volumn', 'meas_step_max': 'measurement_step_max', 'dir_alpha95': 'specimen_alpha95', 'magn_volumne': 'specimen_magn_volumn', 'measurement_step_min': 'meas_step_min', 'meas_step_unit': 'measurement_step_unit', 'dir_dec': 'specimen_dec', 'method_codes': 'magic_method_codes', 'result_quality': 'specimen_flag', 'dir_dang': 'specimen_dang'} 
site3_site2={'int_abs_sigma' : 'site_int_sigma', 'int_abs_sigma_perc' : 'site_int_sigma_perc', 'int_n_samples' : 'site_int_n', 'dir_alpha95' : 'site_alpha95', 'dir_k' : 'site_k', 'dir_n_samples' : 'site_n', 'dir_n_specimens_lines' : 'site_n_lines', 'dir_n_specimens_planes' : 'site_n_planes', 'dir_r' : 'site_r'}
aniso3_aniso2={'specimen':'er_specimen_name', 'aniso_type':'anisotropy_type', 'description':'result_description', 'aniso_ftest':'anisotropy_ftest', 'aniso_ftest12':'anisotropy_ftest12', 'aniso_ftest23':'anisotropy_ftest23', 'aniso_s_mean':'anisotropy_mean', 'aniso_s_n_measurements':'anisotropy_n', 'aniso_s_sigma':'anisotropy_sigma', 'aniso_s_unit':'anisotropy_unit', 'aniso_tilt_correction':'anisotropy_tilt_correction'}

samp3_samp2 = {'int_n_specimens' : 'sample_int_n', 'int_abs_sigma' : 'sample_int_sigma', 'int_abs_sigma_perc' : 'sample_int_sigma_perc', 'dir_alpha95' : 'sample_alpha95', 'dir_n_specimens' : 'sample_n', 'dir_n_specimens_lines' : 'sample_n_lines', 'dir_n_specimens_planes' : 'sample_n_planes', 'dir_k' : 'sample_k', 'dir_r' : 'sample_r'}
meas3_meas2={'treat_dc_field_theta': 'treatment_dc_field_theta', 'sample': 'er_sample_name', 'treat_dc_field': 'treatment_dc_field', 'instrument_codes': 'magic_instrument_codes', 'description': 'measurement_description', 'magn_volume': 'measurement_magn_volume', 'specimen': 'er_specimen_name', 'treat_dc_field_phi': 'treatment_dc_field_phi', 'number': 'measurement_number', 'site': 'er_site_name', 'treat_ac_field': 'treatment_ac_field', 'flag': 'measurement_flag', 'dir_inc': 'measurement_inc', 'location': 'er_location_name', 'dir_dec': 'measurement_dec', 'method_codes': 'magic_method_codes', 'treat_temp': 'treatment_temp', 'magn_moment': 'measurement_magn_moment', 'magn_mass': 'measurement_magn_mass', 'dir_csd': 'measurement_csd'}


print flip_dict(meas3_meas2)


{'treatment_ac_field': 'treat_ac_field', 'measurement_number': 'number', 'treatment_dc_field_theta': 'treat_dc_field_theta', 'er_site_name': 'site', 'er_sample_name': 'sample', 'treatment_dc_field_phi': 'treat_dc_field_phi', 'measurement_magn_volume': 'magn_volume', 'magic_instrument_codes': 'instrument_codes', 'measurement_description': 'description', 'er_location_name': 'location', 'measurement_dec': 'dir_dec', 'measurement_flag': 'flag', 'measurement_magn_moment': 'magn_moment', 'measurement_inc': 'dir_inc', 'treatment_temp': 'treat_temp', 'er_specimen_name': 'specimen', 'measurement_csd': 'dir_csd', 'treatment_dc_field': 'treat_dc_field', 'measurement_magn_mass': 'magn_mass', 'magic_method_codes': 'method_codes'}


In [28]:
NewMeas=[]
# step through records
for rec in data2: NewMeas.append(map_magic.convert_meas('magic3',rec))
pmag.magic_write(WD+'/measurements.txt',NewMeas,'measurements')

25470  records written to file  /Users/ltauxe/Programs/PmagPy/GIT/PmagPy/2_5/McMurdo/measurements.txt


(True, '/Users/ltauxe/Programs/PmagPy/GIT/PmagPy/2_5/McMurdo/measurements.txt')